In [110]:
# IMPORTES
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [111]:
# PASSO 1: Criar o navegador 

nav = webdriver.Chrome()

# Código para que não apareça o navegador enquanto o programa roda
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True
nav = webdriver.Chrome(options=chrome_options)

In [112]:
# PASSO 2: Pegar cotação do dólar

nav.get('https://www.google.com.br/')

# O 'xpath' precisa estar com aspas simples, pois seu texto pode conter aspas duplas
nav.find_element('xpath', 
                 '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input ').send_keys('cotação dólar')

nav.find_element('xpath', 
                 '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input ').send_keys(Keys.ENTER)

dolar = nav.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(dolar)


5.117999999999999


In [113]:
# PASSO 3: Pegar cotação do euro

nav.get('https://www.google.com.br/')

nav.find_element('xpath', 
                 '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input ').send_keys('cotação euro')

nav.find_element('xpath', 
                 '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input ').send_keys(Keys.ENTER)

euro = nav.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(euro)


5.5466999999999995


In [114]:
# PASSO 4: Pegar cotação do ouro

nav.get('https://www.melhorcambio.com/ouro-hoje')

ouro = nav.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')

ouro = ouro.replace(',', '.')

print(ouro)


313.66


In [115]:
# PASSO 5: Fechar o navegador

nav.quit()

In [121]:
# PASSO 6: Atualizar base de dados

dados = pd.read_excel('Produtos.xlsx')

# Atualização da cotação 
dados.loc[dados['Moeda'] == 'Dólar', 'Cotação'] = float(dolar)
dados.loc[dados['Moeda'] == 'Euro', 'Cotação'] = float(euro)
dados.loc[dados['Moeda'] == 'Ouro', 'Cotação'] = float(ouro)

# Atualizando o preço de compra 
dados['Preço de Compra'] = dados['Cotação'] * dados['Preço Original']

# Atualizando o preço de venda
dados['Preço de Venda'] = dados['Preço de Compra'] * dados['Margem']

# Formatação
dados['Cotação'] = dados['Cotação'].map('R$ {:.2f}'.format)
dados['Preço de Compra'] = dados['Preço de Compra'].map('R$ {:.2f}'.format)
dados['Preço de Venda'] = dados['Preço de Venda'].map('R$ {:.2f}'.format)

display(dados)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,R$ 5.12,R$ 5117.95,1.40,R$ 7165.13
1,Carro Renault,4500.00,Euro,R$ 5.55,R$ 24960.15,2.00,R$ 49920.30
2,Notebook Dell,899.99,Dólar,R$ 5.12,R$ 4606.15,1.70,R$ 7830.45
3,IPhone,799.00,Dólar,R$ 5.12,R$ 4089.28,1.70,R$ 6951.78
4,Carro Fiat,3000.00,Euro,R$ 5.55,R$ 16640.10,1.90,R$ 31616.19
5,Celular Xiaomi,480.48,Dólar,R$ 5.12,R$ 2459.10,2.00,R$ 4918.19
6,Joia 20g,20.00,Ouro,R$ 313.66,R$ 6273.20,1.15,R$ 7214.18


In [123]:
# PASSO 7: Exportando a base de dados

dados.to_excel('Produtos Atualizados.xlsx', index=False)